In [1]:
from tqdm import tqdm
import numpy as np
from io import StringIO

In [2]:
import openai

with open("secret/secret_key_file.txt", 'r') as file:
    api_key = file.read().strip()

client = openai.OpenAI(api_key = api_key)

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import re
from bs4 import BeautifulSoup

# Without error capture to ensure the output (whether success or failure) is reported back.
# Attempt to read tables from the HTML file using pandas
file_path = "data/Program - MSR 2022.html"

In [4]:
with open(file_path, 'r') as file:
    html_content = file.read()

In [5]:
soup = BeautifulSoup(html_content, 'lxml')

In [6]:
rows = soup.find_all('tr', class_='hidable')

In [7]:
# Regex to match date pattern (e.g., "Tue 16 May")
date_pattern = re.compile(r"\b(?:Mon|Tue|Wed|Thu|Fri|Sat|Sun)\s\d{1,2}\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)")

In [8]:
extracted_rows = []

# Loop through each row to extract required details
for row in rows:
    temp_row = []
    
    # Extract session details from parent div if not already extracted
    session_div = row.find_previous('div', class_='session-info-in-table')
    session = session_div.get_text(strip=True) if session_div else ''
    
    datetime_info = row.find('td', class_='text-right')
    talk_info = row.find_all('td')[-1]
    
    if datetime_info and talk_info:
        time = datetime_info.find('div', class_='start-time').get_text(strip=True) if datetime_info.find('div', class_='start-time') else ''
        duration = datetime_info.find('strong').get_text(strip=True) if datetime_info.find('strong') else ''
        
        # Calculate end time based on start time and duration if needed
        
        temp_row.append(time)
        # temp_row.append(end_time)  # Calculate and append end time if needed
        temp_row.append(duration)
        
        talk_title = talk_info.find('strong')
        if talk_title:
            temp_row.append(talk_title.get_text(strip=True))
            
            link = talk_title.find('a', href=True)
            temp_row.append(link['href'] if link else '')
        else:
            temp_row.extend(['', ''])
            
        # Extract authors
        authors_div = talk_info.find('div', class_='performers')
        authors = [author.get_text(strip=True) for author in authors_div.find_all('a')] if authors_div else []
        temp_row.append(authors)
        
        # Append session information
        temp_row.append(session)
        
        # Locate date information
        # Tip: You may need to adjust how you locate the 'date information' based on your HTML structure
        date_info = row.find_previous('div', class_='day-wrapper')
        if date_info:
            date_text = date_info.get_text(strip=True)
            matched_date = date_pattern.search(date_text)
            date = matched_date.group(0) if matched_date else "Date Not Found"
        else:
            date = None
        temp_row.append(date)
        
        extracted_rows.append(temp_row)

columns = ['Time', 'Duration', 'Talk Title', 'Link', 'Authors', 'Session', 'Date']
df = pd.DataFrame(extracted_rows, columns=columns)

df.head()

,Time,Duration,Talk Title,Link,Authors,Session,Date
0,22:00,4m,An Empirical Evaluation of GitHub Copilot’s Co...,#,"[Nhan Nguyen, Sarah Nadi]",Session 1Technical Papers/Registered Reportsat...,Tue 17 May
1,22:04,4m,Comments on Comments: Where Code Review and Do...,#,"[Nikitha Rao, Jason Tsay, Martin Hirzel, Vince...",Session 1Technical Papers/Registered Reportsat...,Tue 17 May
2,22:08,7m,Does This Apply to Me? An Empirical Study of T...,#,"[Akalanka Galappaththi, Sarah Nadi, Christoph ...",Session 1Technical Papers/Registered Reportsat...,Tue 17 May
3,22:15,7m,Towards Reliable Agile Iterative Planning via ...,#,"[Jirat Pasuksmit, Patanamon Thongtanunam, Shan...",Session 1Technical Papers/Registered Reportsat...,Tue 17 May
4,22:22,7m,BotHunter: An Approach to Detect Software Bots...,#,"[Ahmad Abdellatif, Mairieli Wessel, Igor Stein...",Session 1Technical Papers/Registered Reportsat...,Tue 17 May


In [9]:
# Convert all column names to lowercase and replace spaces with hyphens
df.columns = [col.lower().replace(' ', '_') for col in df.columns]

# Put date first
df = df[['date'] + [col for col in df.columns if col != 'date']]

# Convert Time to start_time as datetime and calculate end_time
df['start_time'] = pd.to_datetime(df['date'] + ' ' + df['time'], format='%a %d %b %H:%M')
df['duration'] = df['duration'].str.extract('(\d+)').astype(int)  # Extract duration in minutes
df['end_time'] = df['start_time'] + pd.to_timedelta(df['duration'], unit='m')

# Delete the Link column
df = df.drop(['link', 'time'], axis=1)

# Move Authors column to the last position
df = df[[col for col in df.columns if col != 'authors'] + ['authors']]

df = df[['session', 'date', 'start_time', 'end_time', 'duration', 'talk_title','authors']]

# Display the final data frame
df.head()

,session,date,start_time,end_time,duration,talk_title,authors
0,Session 1Technical Papers/Registered Reportsat...,Tue 17 May,1900-05-17 22:00:00,1900-05-17 22:04:00,4,An Empirical Evaluation of GitHub Copilot’s Co...,"[Nhan Nguyen, Sarah Nadi]"
1,Session 1Technical Papers/Registered Reportsat...,Tue 17 May,1900-05-17 22:04:00,1900-05-17 22:08:00,4,Comments on Comments: Where Code Review and Do...,"[Nikitha Rao, Jason Tsay, Martin Hirzel, Vince..."
2,Session 1Technical Papers/Registered Reportsat...,Tue 17 May,1900-05-17 22:08:00,1900-05-17 22:15:00,7,Does This Apply to Me? An Empirical Study of T...,"[Akalanka Galappaththi, Sarah Nadi, Christoph ..."
3,Session 1Technical Papers/Registered Reportsat...,Tue 17 May,1900-05-17 22:15:00,1900-05-17 22:22:00,7,Towards Reliable Agile Iterative Planning via ...,"[Jirat Pasuksmit, Patanamon Thongtanunam, Shan..."
4,Session 1Technical Papers/Registered Reportsat...,Tue 17 May,1900-05-17 22:22:00,1900-05-17 22:29:00,7,BotHunter: An Approach to Detect Software Bots...,"[Ahmad Abdellatif, Mairieli Wessel, Igor Stein..."


In [10]:
df['start_time'] = pd.to_datetime(df['start_time']).astype(int) // 10**9 // 60
df['end_time'] = pd.to_datetime(df['end_time']).astype(int) // 10**9 // 60

df.head()

,session,date,start_time,end_time,duration,talk_title,authors
0,Session 1Technical Papers/Registered Reportsat...,Tue 17 May,-36619320,-36619316,4,An Empirical Evaluation of GitHub Copilot’s Co...,"[Nhan Nguyen, Sarah Nadi]"
1,Session 1Technical Papers/Registered Reportsat...,Tue 17 May,-36619316,-36619312,4,Comments on Comments: Where Code Review and Do...,"[Nikitha Rao, Jason Tsay, Martin Hirzel, Vince..."
2,Session 1Technical Papers/Registered Reportsat...,Tue 17 May,-36619312,-36619305,7,Does This Apply to Me? An Empirical Study of T...,"[Akalanka Galappaththi, Sarah Nadi, Christoph ..."
3,Session 1Technical Papers/Registered Reportsat...,Tue 17 May,-36619305,-36619298,7,Towards Reliable Agile Iterative Planning via ...,"[Jirat Pasuksmit, Patanamon Thongtanunam, Shan..."
4,Session 1Technical Papers/Registered Reportsat...,Tue 17 May,-36619298,-36619291,7,BotHunter: An Approach to Detect Software Bots...,"[Ahmad Abdellatif, Mairieli Wessel, Igor Stein..."


In [11]:
df.shape

(187, 7)

In [12]:
sessions = df.groupby("session").size()
valid_sessions = sessions[sessions>3].index
df = df[df.session.isin(valid_sessions)]

In [13]:
delete_talk_title = "Discussions and Q&A"
df = df[df.talk_title!=delete_talk_title]

In [14]:
df = df[~df.session.str.startswith("Blended")]

In [15]:
df.shape

(125, 7)

In [16]:
df.dtypes

session       object
date          object
start_time     int64
end_time       int64
duration       int64
talk_title    object
authors       object
dtype: object

In [17]:
def merge_intervals(intervals):
    if not intervals:
        return 0
    
    # Sort intervals by the start time
    intervals.sort(key=lambda x: x[0])
    
    merged = [intervals[0]]
    for current_start, current_end in intervals[1:]:
        last_end = merged[-1][1]
        
        if current_start <= last_end:
            # There's an overlap, extend the previous interval
            merged[-1] = (merged[-1][0], max(last_end, current_end))
        else:
            # No overlap, add this interval as is
            merged.append((current_start, current_end))
    
    # Compute total duration in minutes (or another unit as desired)
    total_duration_minutes = sum((end - start) for start, end in merged)   # convert seconds to minutes
    return total_duration_minutes

total_durations = []

for date, group in df.groupby('date'):
    intervals = list(zip(group['start_time'], group['end_time']))
    total_duration = merge_intervals(intervals)
    total_durations.append({'date': date, 'total_duration': total_duration})

# Converting the result into a DataFrame
total_duration_df = pd.DataFrame(total_durations)

total_duration_df

,date,total_duration
0,Fri 20 May,147
1,Thu 19 May,314
2,Tue 17 May,36
3,Wed 18 May,238


In [18]:
dates = df.groupby("date")["duration"].sum().reset_index()


dates

,date,duration
0,Fri 20 May,147
1,Thu 19 May,314
2,Tue 17 May,36
3,Wed 18 May,238


## Get Schedule through GPT


In [19]:
NUM_SESSION_LIST = [2,3,5,10,len(df.session.unique())] # number of sessions we will consider
SESSION_SIZE_LIST = [2, 5, 10] # number of papers in each session. 10 = "All papers".  

In [20]:
N_ITERATIONS = 10 # we want to get confidence intervals

In [21]:
prompt_template = """
Program creation is the process of taking all the accepted papers to a conference and allocating a presentation slot for each paper with parallel sessions. 
The PC chairs of a conference typically do this manually. 
Assign sessions to the following papers based on the following constraints:
1. The total length of all paper presentations within a session should be less than or equal to the length of the session they are in. 
2. No new sessions should be added. 
3. The "Discussions and Q&A" event should only be at the end of a session if present. 
4. All papers must be assigned to some session. 

The output should contain the schedule in the form of the csv representation of a data frame. This csv representation should be in three quotes (```) on both sides so that I can easily extract it from your result and make a data frame. 
Example output format(use as many rows as the actual number of papers):

```
session@talk_title@duration
231@An Empirical Study on Maintainable Method Size in Java@7
223@Improve Quality of Cloud Serverless Architectures through Software Repository Mining@7
15@Extracting corrective actions from code repositories@7
15@How to Improve Deep Learning for Software Analytics (a case study with code smell detection)@7
11@ReCover: a Curated Dataset for Regression Testing Research@4
```

The list of paper and session titles are below: 

Session Lengths:
{sessions_df_string}

Paper durations:
{papers_df_string}


Make sure, above all else, that your response is formatted as requested, with the proper headers.

"""

# Print the start_prompt to verify
print(prompt_template[:500])


Program creation is the process of taking all the accepted papers to a conference and allocating a presentation slot for each paper with parallel sessions. 
The PC chairs of a conference typically do this manually. 
Assign sessions to the following papers based on the following constraints:
1. The total length of all paper presentations within a session should be less than or equal to the length of the session they are in. 
2. No new sessions should be added. 
3. The "Discussions and Q&A" event


In [22]:
def evaluate_schedule(df_human, df_llm):
    sessions_human = df_human.groupby("session")["duration"].sum().sort_values(ascending=False)
    sessions_llm = df_llm.groupby("session")["duration"].sum().sort_values(ascending=False).reset_index()
    sessions_llm.index = pd.Categorical(sessions_llm.index).codes
    del sessions_llm["session"]
    sessions_human.index = pd.Categorical(sessions_human.index).codes
    
    result = {}
    
    result["constraint_num_papers_error"] = len(df_llm) - len(df_human) 
    result["constraint_num_sessions_error"] = len(sessions_llm) - len(sessions_human)
    
    # Iterate and compare values
    for idx, session_duration_llm in sessions_llm.iterrows():
        if idx < len(sessions_human):
            session_duration_human = sessions_human.iloc[idx]
            result[f"constraint_error_session_{idx}_duration"] = session_duration_llm - session_duration_human 
            result[f"constraint_error_session_{idx}_duration_reasonable"] = session_duration_llm / session_duration_human - 1 <= 0.1
        else:
            result[f"constraint_error_session_{idx}_duration"] = np.inf
            result[f"constraint_error_session_{idx}_duration_reasonable"] = False

    
    return result

df_test = df.sample()
evaluate_schedule(df_test, df_test)

{'constraint_num_papers_error': 0,
 'constraint_num_sessions_error': 0,
 'constraint_error_session_0_duration': duration    0
 Name: 0, dtype: int64,
 'constraint_error_session_0_duration_reasonable': duration    True
 Name: 0, dtype: bool}

In [23]:
def get_llm_schedule(df):
    sessions = df.groupby("session")["duration"].sum()
    sessions_sorted = sessions.sort_values(ascending=False)

    prompt = prompt_template.format(
        sessions_df_string = sessions.to_string(index=False)
        ,papers_df_string = df[["talk_title","duration"]].to_string(index=False)
    )

    response = client.chat.completions.create(
    model= "gpt-4-0125-preview", #"gpt-3.5-turbo-0125",  # You can switch this to "gpt-4-turbo-preview", "gpt-3.5-turbo-0125"
        messages=[
            {
                "role": "user",
                "content": prompt
            },
        ],
        temperature=0.8,
        # max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    ans_string = response.choices[0].message.content
    
    match = re.search("```\n(.*?)\n```", ans_string, re.DOTALL)
    if match:
        csv_string = match.group(1)  # Extract the actual CSV data
        # print("Extracted CSV:\n", csv_string)

        # Convert the CSV string to a DataFrame
        # StringIO is used to convert the string to a file-like object
        df_results = pd.read_csv(StringIO(csv_string), sep="@")
        

        # print("Got the df!")
    else:
        print("No CSV data found in the string. Abort!")
        print(ans_string[:200])
        raise ValueError
        
    # print(df_results.head())
    try:
        evaluation = evaluate_schedule(df, df_results)
    except Exception as error:
        print('Caught this error: ' + repr(error))
        print(df_results.head())
        raise ValueError 
        
    return df_results, evaluation


In [24]:
# TESTING CODE TO MAKE SURE THE FUNCTIONS WORK
sampled_sessions = np.random.choice(df['session'].unique(), size=2, replace=False)

# Loop over these selected sessions and sample the required number of papers
session_samples = []
for session in sampled_sessions:
    papers = df[df['session'] == session]
    # If there are fewer papers than the required session_size, we take all available, repeating if necessary
    sampled_papers = papers.sample(n=len(papers)) if len(papers) < 2 else papers.sample(n=2)
    session_samples.append(sampled_papers)

# Now we store the sampled data
df_dataset = pd.concat(session_samples)

test_results = get_llm_schedule(df_dataset)

In [25]:
test_results[0].head()

,session,talk_title,duration
0,14,Using Bandit Algorithms for Selecting Feature ...,7
1,14,To Type or Not to Type? A Systematic Compariso...,7
2,20,Bias in MSR research,10
3,20,Mining Software Repositories in the age of AI,10


In [26]:
test_results[1]

{'constraint_num_papers_error': 0,
 'constraint_num_sessions_error': 0,
 'constraint_error_session_0_duration': duration    0
 Name: 0, dtype: int64,
 'constraint_error_session_0_duration_reasonable': duration    True
 Name: 0, dtype: bool,
 'constraint_error_session_1_duration': duration    0
 Name: 1, dtype: int64,
 'constraint_error_session_1_duration_reasonable': duration    True
 Name: 1, dtype: bool}

In [27]:
all_results = {}
for num_sessions in tqdm(NUM_SESSION_LIST, desc='Num Sessions Progress'):
    for session_size in tqdm(SESSION_SIZE_LIST, desc='Session Size Progress', leave=False):
        sampled_sessions = np.random.choice(df['session'].unique(), size=num_sessions, replace=False)

        # Loop over these selected sessions and sample the required number of papers
        session_samples = []
        for session in sampled_sessions:
            papers = df[df['session'] == session]
            # If there are fewer papers than the required session_size, we take all available, repeating if necessary
            sampled_papers = papers.sample(n=len(papers)) if len(papers) < session_size else papers.sample(n=session_size)
            session_samples.append(sampled_papers)

        # Now we store the sampled data
        df_dataset = pd.concat(session_samples)

        for iteration in range(N_ITERATIONS):
            try:
                all_results[(num_sessions, session_size, iteration)] = df_dataset, get_llm_schedule(df_dataset), 1
            except ValueError: # try once more
                try:
                    all_results[(num_sessions, session_size, iteration)] = df_dataset, get_llm_schedule(df_dataset), 1
                except ValueError: # Give up and report failure. 
                    all_results[(num_sessions, session_size,iteration)] = df_dataset, {}, 0
                    print(f"Give up this one, {(num_sessions, session_size, iteration)}")
            


Num Sessions Progress: 100%|██████████| 5/5 [49:43<00:00, 596.75s/it]


In [43]:
# all_results_backup = all_results.copy()

In [30]:
def extract_data(all_results):
    extracted_data = []
    for key, (df_dataset, evaluation, success) in all_results.items():
        if success:
            num_sessions, session_size, iteration = key
            # Extracting data specifically related to `constraint_error_session_...`
            constraint_errors = {k: v for k, v in evaluation[1].items() if k.startswith('constraint_error_session_')}
            entry = {
                'num_sessions': num_sessions,
                'session_size': session_size,
                'iteration': iteration,
                'constraint_errors': constraint_errors
            }
            extracted_data.append(entry)
    return extracted_data

# Assuming `all_results` is already populated, we call the function
extracted_data = extract_data(all_results)
df_extracted_data = pd.DataFrame(extracted_data)

# Optionally save or work further with `df_extracted_data`
df_extracted_data.head()

,num_sessions,session_size,iteration,constraint_errors
0,2,2,0,"{'constraint_error_session_0_duration': [-3], ..."
1,2,2,1,"{'constraint_error_session_0_duration': [-3], ..."
2,2,2,2,"{'constraint_error_session_0_duration': [-3], ..."
3,2,2,3,"{'constraint_error_session_0_duration': [-3], ..."
4,2,2,4,"{'constraint_error_session_0_duration': [-3], ..."


In [45]:
df_extracted_data["constraint_errors"].iloc[0]

{'constraint_error_session_0_duration': duration   -3
 Name: 0, dtype: int64,
 'constraint_error_session_0_duration_reasonable': duration    True
 Name: 0, dtype: bool,
 'constraint_error_session_1_duration': duration    3
 Name: 1, dtype: int64,
 'constraint_error_session_1_duration_reasonable': duration    False
 Name: 1, dtype: bool}

In [37]:
def calculate_sum_absolute_error(df_extracted_data):
    # Temporary container to accumulate data
    results = []

    # Iterate over rows of the dataframe
    for index, row in df_extracted_data.iterrows():
        # Sum and count for calculating mean
        sum_errors = 0
        count = 0

        # Extract error entries
        error_dict = row['constraint_errors']
        for key, value in error_dict.items():
            if 'duration' in key:
                # Calculate absolute error and update sum and count
                sum_errors += abs(value[0])  # Assuming the error value is encapsulated in a list
                count += 1

        if count > 0:
            sum_error = sum_errors 
        else:
            sum_error = None

        results.append({
            'num_sessions': row['num_sessions'],
            'session_size': row['session_size'],
            'iteration': row['iteration'],
            'sum_absolute_error': sum_error
        })

    return pd.DataFrame(results)

def calculate_percentage_of_unreasonable(df_extracted_data):
    # Temporary container to accumulate data
    results = []

    # Iterate over rows of the dataframe
    for index, row in df_extracted_data.iterrows():
        # Count entries for calculating percentage
        total = 0
        not_reasonable_count = 0

        # Extract error entries
        error_dict = row['constraint_errors']
        for key, value in error_dict.items():
            if 'reasonable' in key:
                # Check reasonable flag
                is_reasonable = value[0]  # Assuming the value is encapsulated in a list
                if not is_reasonable:
                    not_reasonable_count += 1
                total += 1

        if total > 0:
            percentage_unreasonable = (not_reasonable_count / total) * 100
        else:
            percentage_unreasonable = None

        results.append({
            'num_sessions': row['num_sessions'],
            'session_size': row['session_size'],
            'iteration': row['iteration'],
            'percentage_unreasonable': percentage_unreasonable
        })

    return pd.DataFrame(results)

In [38]:
# Apply these functions to the extracted data
df_sum_errors = calculate_sum_absolute_error(df_extracted_data)
df_percentage_unreasonable = calculate_percentage_of_unreasonable(df_extracted_data)


/var/tmp/ipykernel_3513670/3637206885.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sum_errors += abs(value[0])  # Assuming the error value is encapsulated in a list
/var/tmp/ipykernel_3513670/3637206885.py:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  is_reasonable = value[0]  # Assuming the value is encapsulated in a list


In [41]:
df_sum_errors.groupby(["num_sessions","session_size"])["sum_absolute_error"].mean()

num_sessions  session_size
2             2                 7.0
              5                 4.0
              10                4.0
3             2                 4.8
              5                 5.0
              10               10.1
5             2                28.4
              5                25.0
              10               34.4
10            2                48.2
              5                34.6
              10               54.1
20            2                45.6
              5               114.0
              10              115.1
Name: sum_absolute_error, dtype: float64

In [44]:
df_percentage_unreasonable.groupby(["num_sessions","session_size"])["percentage_unreasonable"].mean()

num_sessions  session_size
2             2               45.000000
              5                0.000000
              10               0.000000
3             2               10.000000
              5               16.666667
              10               3.333333
5             2               66.666667
              5               26.000000
              10              40.666667
10            2               70.722222
              5               12.000000
              10               1.000000
20            2               54.066176
              5                8.026316
              10               9.500000
Name: percentage_unreasonable, dtype: float64

### Look at one example

In [47]:
all_results[(2,2,0)][0] # true schedule

,session,date,start_time,end_time,duration,talk_title,authors
97,Vision & Reflections Track: FutureTechnical Pa...,Thu 19 May,-36616970,-36616960,10,The Next Generation of Software Developers,[Denae Ford]
96,Vision & Reflections Track: FutureTechnical Pa...,Thu 19 May,-36616980,-36616970,10,Bias in MSR research,[Alexander Serebrenik]
103,Session 12: Integration & Large-Scale MiningTe...,Thu 19 May,-36616549,-36616542,7,Methods for Stabilizing Models across Large Sa...,"[Suvodeep Majumder, Tianpei Xia, Rahul Krishna..."
105,Session 12: Integration & Large-Scale MiningTe...,Thu 19 May,-36616535,-36616528,7,FaST: A linear time stack trace alignment heur...,"[Irving Muller Rodrigues, Daniel Aloise, Erald..."


In [49]:
all_results[(2,2,0)][1][0] # generated schedule

,session,talk_title,duration
0,14,The Next Generation of Software Developers,10
1,14,FaST: A linear time stack trace alignment heur...,7
2,20,Bias in MSR research,10
3,20,Methods for Stabilizing Models across Large Sa...,7


In [60]:
(df_sum_errors.groupby(["num_sessions","session_size"])["sum_absolute_error"].mean()/2)

num_sessions  session_size
2             2                3.50
              5                2.00
              10               2.00
3             2                2.40
              5                2.50
              10               5.05
5             2               14.20
              5               12.50
              10              17.20
10            2               24.10
              5               17.30
              10              27.05
20            2               22.80
              5               57.00
              10              57.55
Name: sum_absolute_error, dtype: float64

In [61]:
df_cell.head()

num_sessions  session_size
2             2               45.000000
              5                0.000000
              10               0.000000
3             2               10.000000
              5               16.666667
Name: percentage_unreasonable, dtype: float64

In [67]:
df_cell = df_percentage_unreasonable.groupby(["num_sessions","session_size"])["percentage_unreasonable"].mean() #.reset_index()

# df_cell = df_cell.join(df_sum_errors.groupby(["num_sessions","session_size"])["sum_absolute_error"].mean()/2).reset_index()

df_cell = pd.concat([df_sum_errors.groupby(["num_sessions","session_size"])["sum_absolute_error"].mean()/2, df_cell], axis=1)

df_cell.iloc[:,-1] *= df_cell.iloc[:,0] / 100

df_cell.iloc[:,-2:] = df_cell.iloc[:,-2:].round(2)

df_cell.reset_index()

,num_sessions,session_size,sum_absolute_error,percentage_unreasonable
0,2,2,3.50,1.58
1,2,5,2.00,0.00
2,2,10,2.00,0.00
3,3,2,2.40,0.24
4,3,5,2.50,0.42
5,3,10,5.05,0.17
6,5,2,14.20,9.47
7,5,5,12.50,3.25
8,5,10,17.20,6.99
9,10,2,24.10,17.04
